## Step 1: 
Prompt Gen Prompt + Prompt Gen Test cases + API trả ra cả Prompt và Test Cases của 2 Prompt kia - --- VỚi đầu vào là: format_output, samples, conditions

## Step 2
- từ file evaluator.py viết thành các file 
+, file run_prompt hàng loạt với bất kỳ prompt nào 
+, file run_prompt_with_test cases generated.py ? (kế thừa từ run_prompt.py và nhận input là: Prompt generated và testcases Generated. 
+, file run_prompt_evaluate: expected_output của test cases với output của run_prompt_with_test cases generated.py

Tách thành api run: run_prompt_with_test cases generated
Và api run: un_prompt_evaluate: expected_output của test cases với output của run_prompt_with_test cases generated

# Frontend 

Tôi sẽ tạo frontend React để tương tác với backend API. Cấu trúc như sau:

```
frontend/
├── src/
│   ├── components/
│   │   ├── PromptRunner/
│   │   │   ├── PromptRunner.tsx      # Component chạy prompt
│   │   │   └── PromptRunner.css
│   │   ├── ResultEvaluator/  
│   │   │   ├── ResultEvaluator.tsx   # Component đánh giá kết quả
│   │   │   └── ResultEvaluator.css
│   │   └── TestCaseViewer/
│   │       ├── TestCaseViewer.tsx    # Component hiển thị test cases
│   │       └── TestCaseViewer.css
│   ├── services/
│   │   └── api.ts                    # API calls
│   ├── types/
│   │   └── index.ts                  # TypeScript interfaces
│   ├── App.tsx
│   └── index.tsx
├── package.json
└── tsconfig.json
```

1. Đầu tiên là types để map với backend models:

```typescript:frontend/src/types/index.ts
export interface TestCase {
  input: string;
  expected_output: string;
  prompt_output: string;
  is_correct: boolean;
  similarity_score: number;
}

export interface RunPromptRequest {
  prompt: string;
  test_cases: TestCase[];
}

export interface RunPromptResponse {
  test_cases: TestCase[];
  total_time: number;
}

export interface EvaluateResultsRequest {
  test_cases: TestCase[];
}

export interface EvaluateResultsResponse {
  accuracy: number;
  avg_similarity: number;
  test_cases: TestCase[];
}
```

2. Service để gọi API:

```typescript:frontend/src/services/api.ts
import axios from 'axios';
import { RunPromptRequest, RunPromptResponse, EvaluateResultsRequest, EvaluateResultsResponse } from '../types';

const API_URL = 'http://localhost:25043';

export const runPrompt = async (request: RunPromptRequest): Promise<RunPromptResponse> => {
  const response = await axios.post(`${API_URL}/api/run-prompt`, request);
  return response.data;
};

export const evaluateResults = async (request: EvaluateResultsRequest): Promise<EvaluateResultsResponse> => {
  const response = await axios.post(`${API_URL}/api/evaluate-results`, request);
  return response.data;
};
```

3. Component chạy prompt:

```typescript:frontend/src/components/PromptRunner/PromptRunner.tsx
import React, { useState } from 'react';
import { TestCase, RunPromptRequest } from '../../types';
import { runPrompt } from '../../services/api';
import './PromptRunner.css';

interface Props {
  onResults: (testCases: TestCase[]) => void;
}

export const PromptRunner: React.FC<Props> = ({ onResults }) => {
  const [prompt, setPrompt] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();
    setLoading(true);
    setError('');

    try {
      // Sample test cases
      const testCases: TestCase[] = [
        {
          input: "Đánh giá phát âm: 'hello'",
          expected_output: "Phát âm chuẩn",
          prompt_output: "",
          is_correct: false,
          similarity_score: 0
        },
        {
          input: "Đánh giá phát âm: 'world'",
          expected_output: "Phát âm không chuẩn",
          prompt_output: "",
          is_correct: false,
          similarity_score: 0
        }
      ];

      const request: RunPromptRequest = {
        prompt,
        test_cases: testCases
      };

      const response = await runPrompt(request);
      onResults(response.test_cases);
    } catch (err) {
      setError('Failed to run prompt');
      console.error(err);
    } finally {
      setLoading(false);
    }
  };

  return (
    <div className="prompt-runner">
      <h2>Run Prompt</h2>
      <form onSubmit={handleSubmit}>
        <div className="form-group">
          <label>Prompt:</label>
          <textarea
            value={prompt}
            onChange={(e) => setPrompt(e.target.value)}
            placeholder="Enter your prompt here..."
            rows={5}
          />
        </div>
        <button type="submit" disabled={loading}>
          {loading ? 'Running...' : 'Run Prompt'}
        </button>
      </form>
      {error && <div className="error">{error}</div>}
    </div>
  );
};
```

4. Component hiển thị và đánh giá kết quả:

```typescript:frontend/src/components/ResultEvaluator/ResultEvaluator.tsx
import React, { useState } from 'react';
import { TestCase, EvaluateResultsResponse } from '../../types';
import { evaluateResults } from '../../services/api';
import './ResultEvaluator.css';

interface Props {
  testCases: TestCase[];
  onEvaluated: (results: EvaluateResultsResponse) => void;
}

export const ResultEvaluator: React.FC<Props> = ({ testCases, onEvaluated }) => {
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  const handleEvaluate = async () => {
    setLoading(true);
    setError('');

    try {
      const results = await evaluateResults({ test_cases: testCases });
      onEvaluated(results);
    } catch (err) {
      setError('Failed to evaluate results');
      console.error(err);
    } finally {
      setLoading(false);
    }
  };

  return (
    <div className="result-evaluator">
      <h2>Test Results</h2>
      <div className="test-cases">
        {testCases.map((testCase, index) => (
          <div key={index} className="test-case">
            <div>Input: {testCase.input}</div>
            <div>Expected: {testCase.expected_output}</div>
            <div>Got: {testCase.prompt_output}</div>
            <div>Correct: {testCase.is_correct ? '✅' : '❌'}</div>
            <div>Similarity: {testCase.similarity_score.toFixed(2)}</div>
          </div>
        ))}
      </div>
      <button onClick={handleEvaluate} disabled={loading}>
        {loading ? 'Evaluating...' : 'Evaluate Results'}
      </button>
      {error && <div className="error">{error}</div>}
    </div>
  );
};
```

5. App component để tích hợp:

```typescript:frontend/src/App.tsx
import React, { useState } from 'react';
import { PromptRunner } from './components/PromptRunner/PromptRunner';
import { ResultEvaluator } from './components/ResultEvaluator/ResultEvaluator';
import { TestCase, EvaluateResultsResponse } from './types';
import './App.css';

function App() {
  const [testCases, setTestCases] = useState<TestCase[]>([]);
  const [evaluationResults, setEvaluationResults] = useState<EvaluateResultsResponse | null>(null);

  return (
    <div className="app">
      <h1>Auto-Prompting Tool</h1>
      
      <div className="container">
        <div className="section">
          <PromptRunner onResults={setTestCases} />
        </div>

        {testCases.length > 0 && (
          <div className="section">
            <ResultEvaluator 
              testCases={testCases}
              onEvaluated={setEvaluationResults}
            />
          </div>
        )}

        {evaluationResults && (
          <div className="section results">
            <h2>Evaluation Results</h2>
            <div>Accuracy: {(evaluationResults.accuracy * 100).toFixed(1)}%</div>
            <div>Average Similarity: {(evaluationResults.avg_similarity * 100).toFixed(1)}%</div>
          </div>
        )}
      </div>
    </div>
  );
}

export default App;
```

6. Styles:

```css:frontend/src/App.css
.app {
  padding: 20px;
  max-width: 1200px;
  margin: 0 auto;
}

.container {
  display: flex;
  flex-direction: column;
  gap: 20px;
}

.section {
  padding: 20px;
  border: 1px solid #ddd;
  border-radius: 8px;
  background: white;
}

.results {
  background: #f5f5f5;
}
```

Cần tôi giải thích thêm phần nào không?


Oke nha, giờ viết Tài liệu nha
1. Kiến trúc và luồng Backend
2. Tài liệu API nhá. 
3. ....
...

Các tài liệu khác cần thiết 

- viết 1 file .sh để check port 25043, 25044 trước đó và dừng nếu cần
- sudo docker compose up --build -d , còn lại ko dùng sudo